In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

### lecture des données

In [2]:
data = pd.read_csv('../Data/Train.csv', parse_dates=['datetime'])
weather = pd.read_csv('../Data/Weather_Nairobi_Daily_GFS.csv', parse_dates=['Date'])
road_segment_locs = gpd.read_file('../segments_geometry.geojson')
road_surveys = pd.read_csv('../Data/Segment_info.csv')

In [105]:
int(data.shape[0]*0.72)*3

13644

###  merge weather + data

In [3]:
def mergeWeatherData(weather,data):
    data['Date'] = [d.date() for d in data['datetime']]
    data['time'] = [d.time() for d in data['datetime']]
    data['Date'] = data['Date'].astype('datetime64[ns]')
    data = data.drop(['uid'],axis=1)
    data_train = pd.merge(data, weather, how='left', on='Date')
    data_train = data_train.set_index('datetime')
    return data_train

In [4]:
data_merged = mergeWeatherData(weather,data)

In [5]:
data_merged.head()

,latitude,longitude,Date,time,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground
datetime,,,,,,,,,,
2018-01-01 00:25:46,-1.188850,36.931382,2018-01-01,00:25:46,24.1,72.700005,0.00956,15.161493,-0.012186,-0.933914
2018-01-01 02:02:39,-0.662939,37.208730,2018-01-01,02:02:39,24.1,72.700005,0.00956,15.161493,-0.012186,-0.933914
2018-01-01 02:31:49,-0.662939,37.208730,2018-01-01,02:31:49,24.1,72.700005,0.00956,15.161493,-0.012186,-0.933914
2018-01-01 03:04:01,-1.288087,36.826583,2018-01-01,03:04:01,24.1,72.700005,0.00956,15.161493,-0.012186,-0.933914
2018-01-01 03:58:49,-1.188850,36.931382,2018-01-01,03:58:49,24.1,72.700005,0.00956,15.161493,-0.012186,-0.933914


In [6]:
data_merged.dtypes

latitude                                       float64
longitude                                      float64
Date                                    datetime64[ns]
time                                            object
precipitable_water_entire_atmosphere           float64
relative_humidity_2m_above_ground              float64
specific_humidity_2m_above_ground              float64
temperature_2m_above_ground                    float64
u_component_of_wind_10m_above_ground           float64
v_component_of_wind_10m_above_ground           float64
dtype: object

In [7]:
data_merged.shape

(6318, 10)

In [8]:
data_merged = data_merged.dropna()

### merge data_merged + road_survey

In [61]:
def getNeighbourSegment(location, segments):
    result = 0
    dists = []
    for segment in segments:
        dist = []
        for loc in segment:
            d = ((location[0] - loc[0])**2 + (location[1] - location[1])**2)**0.5
            dist.append(d)
        dists.append(min(dist))
    return np.argmin(dists)

array([[36.8260292 , -1.2844709 ],
       [36.82604955, -1.2844441 ],
       [36.82607659, -1.28440849],
       [36.82610364, -1.28437287],
       [36.82613068, -1.28433725],
       [36.82615773, -1.28430164],
       [36.82618477, -1.28426602],
       [36.82621182, -1.2842304 ],
       [36.82623886, -1.28419479],
       [36.82626591, -1.28415917],
       [36.82629295, -1.28412356],
       [36.82631926, -1.28408742],
       [36.82634394, -1.28405012],
       [36.82636862, -1.28401283],
       [36.8263933 , -1.28397553],
       [36.82641797, -1.28393824],
       [36.82644161, -1.2839003 ],
       [36.82646403, -1.2838616 ],
       [36.82648644, -1.2838229 ],
       [36.82650886, -1.2837842 ],
       [36.82653127, -1.28374551],
       [36.82655561, -1.28370805]])

In [10]:
segments_merged = pd.merge(road_segment_locs, road_surveys, on='segment_id', how='left')

In [11]:
segments_merged.head()

,segment_id,road_name,geometry,side,25_48,79_76,79_65,79_39,28_78_50,55_90,...,80_97_61_65_11,80_97_61_65_12,80_97_61_65_15,80_97_61_65_16,80_97_61_39_0,80_97_61_39_10,80_97_61_39_11,80_97_61_39_12,80_97_61_39_15,80_97_61_39_16
0,28_1,accra road-tertiary,"LINESTRING (36.82603 -1.28447, 36.82605 -1.284...",1,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,28_1,accra road-tertiary,"LINESTRING (36.82603 -1.28447, 36.82605 -1.284...",2,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,137_1,accra road-tertiary,"LINESTRING (36.82656 -1.28371, 36.82658 -1.283...",1,1,0,4,4,0.0,5,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,137_1,accra road-tertiary,"LINESTRING (36.82656 -1.28371, 36.82658 -1.283...",2,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,137_2,accra road-tertiary,"LINESTRING (36.82711 -1.28289, 36.82713 -1.282...",1,1,0,4,4,0.0,5,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [66]:
segments_merged.columns.drop('geometry')

Index(['segment_id', 'road_name', 'side', '25_48', '79_76', '79_65', '79_39',
       '28_78_50', '55_90', '26_76',
       ...
       '80_97_61_65_11', '80_97_61_65_12', '80_97_61_65_15', '80_97_61_65_16',
       '80_97_61_39_0', '80_97_61_39_10', '80_97_61_39_11', '80_97_61_39_12',
       '80_97_61_39_15', '80_97_61_39_16'],
      dtype='object', length=229)

In [86]:
def getDataTrain(data_merged,road_data):
    column_names = list(data_merged.columns) + list(road_data.columns[4:]) + ['segment_id'] + ['side'] +['accident'] 
    data_train = pd.DataFrame(columns = column_names)
    segments = road_segment_locs['geometry'].apply(lambda x: np.array(x)).values
    data_range = range(data_merged.shape[0])
    length = data_merged.shape[0]
    for i in data_range:
        print(f'{(i/length)*100}%')
        segment = getNeighbourSegment([data_merged.iloc[i]['longitude'],data_merged.iloc[i]['latitude']],segments)
        d = {}
        for column in data_merged.columns:
            d[column] = data_merged.iloc[i][column]
        for column in road_data.columns[4:]:
            d[column] = road_data.iloc[segment][column]
        segment_id = road_data['segment_id'][segment]
        segment_side = road_data['side'][segment]
        d['side'] = segment_side
        d['segment_id'] = segment_id
        d['accident'] = 1
        data_train = data_train.append(d, ignore_index=True)
        ## les segment qui n'ont pas d'accident
        segments_range = range(segments.shape[0]);
        for s in segments_range:
            d = {}
            if s != segment:
                s_id = road_data['segment_id'][s]
                side = road_data['side'][s]
                d['side'] = side
                d['segment_id'] = s_id
                d['accident'] = 0
                d['longitude'] = segments[s][segments[s].shape[0]//2][0]
                d['latitude'] = segments[s][segments[s].shape[0]//2][1]
                ##date,time,weather
                for column in data_merged.columns.drop(['longitude','latitude']):
                    d[column] = data_merged.iloc[i][column]
                ##road_survey
                for column in road_data.columns[4:]:
                    d[column] = road_data.iloc[s][column]
                data_train = data_train.append(d, ignore_index = True)
    return data_train

In [88]:
train_data = getDataTrain(data_merged,segments_merged)

0.0%
0.01590583744234134%
0.03181167488468268%
0.04771751232702402%
0.06362334976936536%
0.0795291872117067%
0.09543502465404805%
0.11134086209638938%
0.12724669953873072%


KeyboardInterrupt: 

In [85]:
train_data.shape

(792, 239)

In [77]:
segments[1][segments[1].shape[0]//2]

array([36.82684172, -1.28330789])

In [16]:
segments = road_segment_locs['geometry'].apply(lambda x: np.array(x)).values


array([array([[36.8260292 , -1.2844709 ],
       [36.82604955, -1.2844441 ],
       [36.82607659, -1.28440849],
       [36.82610364, -1.28437287],
       [36.82613068, -1.28433725],
       [36.82615773, -1.28430164],
       [36.82618477, -1.28426602],
       [36.82621182, -1.2842304 ],
       [36.82623886, -1.28419479],
       [36.82626591, -1.28415917],
       [36.82629295, -1.28412356],
       [36.82631926, -1.28408742],
       [36.82634394, -1.28405012],
       [36.82636862, -1.28401283],
       [36.8263933 , -1.28397553],
       [36.82641797, -1.28393824],
       [36.82644161, -1.2839003 ],
       [36.82646403, -1.2838616 ],
       [36.82648644, -1.2838229 ],
       [36.82650886, -1.2837842 ],
       [36.82653127, -1.28374551],
       [36.82655561, -1.28370805]]),
       array([[36.82655561, -1.28370805],
       [36.82658162, -1.28367167],
       [36.82660763, -1.28363529],
       [36.82663364, -1.28359891],
       [36.82665965, -1.28356254],
       [36.82668566, -1.28352616],
    

In [77]:
segments_merged.head()

,segment_id,road_name,geometry,side,25_48,79_76,79_65,79_39,28_78_50,55_90,...,80_97_61_65_11,80_97_61_65_12,80_97_61_65_15,80_97_61_65_16,80_97_61_39_0,80_97_61_39_10,80_97_61_39_11,80_97_61_39_12,80_97_61_39_15,80_97_61_39_16
0,28_1,accra road-tertiary,"LINESTRING (36.82603 -1.28447, 36.82605 -1.284...",1,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,28_1,accra road-tertiary,"LINESTRING (36.82603 -1.28447, 36.82605 -1.284...",2,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,137_1,accra road-tertiary,"LINESTRING (36.82656 -1.28371, 36.82658 -1.283...",1,1,0,4,4,0.0,5,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,137_1,accra road-tertiary,"LINESTRING (36.82656 -1.28371, 36.82658 -1.283...",2,1,0,4,4,0.0,5,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,137_2,accra road-tertiary,"LINESTRING (36.82711 -1.28289, 36.82713 -1.282...",1,1,0,4,4,0.0,5,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
